In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()
tf.logging.set_verbosity(tf.logging.ERROR)

import warnings
warnings.filterwarnings('ignore')

from sklearn.compose import make_column_transformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import OneHotEncoder

from aif360.sklearn.preprocessing import Reweighing, ReweighingMeta, FairAdapt, LearnedFairRepresentations
from aif360.sklearn.inprocessing import AdversarialDebiasing, ExponentiatedGradientReduction, GridSearchReduction
from aif360.sklearn.postprocessing import CalibratedEqualizedOdds, RejectOptionClassifier, PostProcessingMeta, RejectOptionClassifierCV
from aif360.sklearn.datasets import fetch_adult
from aif360.sklearn.metrics import disparate_impact_ratio, average_odds_error, generalized_fpr
from aif360.sklearn.metrics import generalized_fnr, difference

# from glob import glob
# import json,os

In [4]:
import json,os
from glob import glob
import traceback


def writejson(path,arr,name):
    try:
        os.mkdir(path+'preds/')
    except:
        pass
    f = open(path+'preds/'+name,'w')
    json.dump(arr.tolist(),f)
    f.close()

In [5]:
path = 'data/datasets/publiccov_ny/'
synthfols = glob("data/datasets/publiccov_ny/synthetic/*/")
paths = [path]
paths.extend(synthfols)

path = 'data/datasets/publiccov_tx/'
synthfols = glob("data/datasets/publiccov_tx/synthetic/*/")
paths.append(path)
paths.extend(synthfols)

In [6]:
def task(path):
    
#     if os.path.exists(path+'preds/baseline_pred.json') == False:
    print(path)

    train_df = pd.read_csv(path+'train.csv',header=None)

    train_df.columns = ['AGEP', 'SCHL', 'MAR', 'SEX', 'DIS', 'ESP', 'CIT', 'MIG', 'MIL', 'ANC',
       'NATIVITY', 'DEAR', 'DEYE', 'DREM', 'PINCP', 'ESR', 'FER', 'RAC1P',
       'PUBCOV']


    train_df.index = train_df['SEX']

    train_df = train_df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

    test_df = pd.read_csv(path+'test.csv',header=None)

    test_df.columns = ['AGEP', 'SCHL', 'MAR', 'SEX', 'DIS', 'ESP', 'CIT', 'MIG', 'MIL', 'ANC',
       'NATIVITY', 'DEAR', 'DEYE', 'DREM', 'PINCP', 'ESR', 'FER', 'RAC1P',
       'PUBCOV']


    test_df.index = test_df['SEX']

    test_df = test_df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
    


    X_train = train_df.drop(['PUBCOV'],axis=1)
    X_test = test_df.drop(['PUBCOV'],axis=1)

    y_train = pd.Series(train_df['PUBCOV'])
    y_test = pd.Series(test_df['PUBCOV'])

    y_train = pd.Series(y_train.factorize(sort=True)[0], index=y_train.index)
    y_test = pd.Series(y_test.factorize(sort=True)[0], index=y_test.index)

    X_merged = pd.concat([X_train,X_test])

    ohe = make_column_transformer(
        (OneHotEncoder(sparse=False), X_merged.dtypes == 'object'),
        remainder='passthrough', verbose_feature_names_out=False)

    X_merged_temp  = pd.DataFrame(ohe.fit_transform(X_merged), columns=ohe.get_feature_names_out(), index=X_merged.index)

    X_train  = pd.DataFrame(ohe.transform(X_train), columns=ohe.get_feature_names_out(), index=X_train.index)
    X_test = pd.DataFrame(ohe.transform(X_test), columns=ohe.get_feature_names_out(), index=X_test.index)
    

    #### BASELINE
    try:
        y_pred_baseline = LogisticRegression(solver='liblinear').fit(X_train, y_train).predict(X_test)
        writejson(path,y_pred_baseline,'baseline_pred.json')    
    except:
        traceback.print_exc()
    

    #### Adv. Reweighting (Preproc.)
    
    try:
        rew = ReweighingMeta(estimator=LogisticRegression(solver='liblinear'),
                     reweigher=Reweighing('SEX'))
        params = {'estimator__C': [1, 10]}

        clf = GridSearchCV(rew, params, scoring='accuracy', cv=5)
        clf.fit(X_train, y_train)
        y_pred_advrew = clf.predict(X_test)
        writejson(path,y_pred_advrew,'adv_rew_pred.json')  
    except:
        traceback.print_exc()

    

    #### LearnedFairRepresentations (Preproc.)
    
    try:
        LFR = LearnedFairRepresentations(prot_attr='SEX')
        LFR.fit(X_train, y_train)

        y_pred_lfr = LFR.predict(X_test)
        writejson(path,y_pred_lfr,'lfr_pred.json')  
    except:
        traceback.print_exc()

    #### Adv Deb. (Inproc.)
    
    try:
        adv_deb = AdversarialDebiasing(prot_attr='SEX')
        adv_deb.fit(X_train, y_train)
        adv_deb.score(X_test, y_test)

        y_pred_adv_deb = adv_deb.predict(X_test)
        writejson(path,y_pred_adv_deb,'adv_deb_pred.json')

        adv_deb.sess_.close() 
    except:
        traceback.print_exc()

    

    #### ExponentiatedGradientReduction. (Inproc.)
    
    try:
        consts = ['EqualizedOdds']#['DemographicParity','EqualizedOdds','TruePositiveRateParity','FalsePositiveRateParity','ErrorRateParity']
        for const in consts:
            EGR = ExponentiatedGradientReduction(prot_attr=['SEX_Female','SEX_Male'], estimator=LogisticRegression(solver='liblinear'), constraints = const)
            EGR.fit(X_train, y_train)
            print(const,EGR.score(X_test, y_test))
            y_pred_egr = EGR.predict(X_test)
            writejson(path,y_pred_egr,'egr_pred_'+const+'.json')
    except:
        traceback.print_exc()

    #### GridSearchReduction. (Inproc.)

    try:
        consts = ['EqualizedOdds']#['DemographicParity','EqualizedOdds','TruePositiveRateParity','FalsePositiveRateParity','ErrorRateParity']
        for const in consts:
            GSR = GridSearchReduction(prot_attr=['SEX_Female','SEX_Male'], estimator=LogisticRegression(solver='liblinear'), constraints = const)
            GSR.fit(X_train, y_train)
            print(const,GSR.score(X_test, y_test))
            y_pred_gsr = GSR.predict(X_test)
            writejson(path,y_pred_gsr,'gsr_pred_'+const+'.json')
    except:
        traceback.print_exc()



    X_train_temp = X_train.set_index(['SEX_Male'], drop = False)
    y_train_temp = y_train.copy()
    y_train_temp.index = X_train_temp.index

    X_test_temp = X_test.set_index(['SEX_Male'], drop = False)
    y_test_temp = y_test.copy()
    y_test_temp.index = X_test_temp.index

    #### Cal. Eq.Odds (Postproc.)

    try:
        consts = ['weighted']#['fnr','fpr','weighted']
        for const in consts:
            cal_eq_odds = CalibratedEqualizedOdds(prot_attr='SEX_Male', cost_constraint=const)
            postproc = PostProcessingMeta(estimator=LogisticRegression(solver='liblinear'), postprocessor=cal_eq_odds)
            postproc.fit(X_train_temp, y_train_temp)
            print(accuracy_score(y_test_temp, postproc.predict(X_test_temp)))
            y_pred_caleq = postproc.predict(X_test_temp)
            writejson(path,y_pred_caleq,'caleq_pred_'+const+'.json')
    except:
        traceback.print_exc()

    #### RejectOptionClassifier (Postproc.)

    try:
        consts = ['average_odds']#['statistical_parity', 'average_odds', 'equal_opportunity']
        for const in consts:
            rocv = PostProcessingMeta(LogisticRegression(solver='liblinear'), RejectOptionClassifierCV('SEX_Male', scoring=const))
            rocv.fit(X_train_temp, y_train_temp)
            print(accuracy_score(y_test_temp, rocv.predict(X_test_temp)))
            y_pred_rocv = rocv.predict(X_test_temp)
            writejson(path,y_pred_rocv,'rocv_pred_'+const+'.json')
    except:
        traceback.print_exc()  
    
    return path,"Done"


In [7]:
# task(path)

In [ ]:
for path in paths:
    try:
        task(path)
    except:
        traceback.print_exc()

data/datasets/publiccov_ny/


2023-02-15 16:04:47.009927: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-15 16:04:48.507581: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46704 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:b1:00.0, compute capability: 8.6
2023-02-15 16:04:48.508274: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 46704 MB memory:  -> device: 1, name: NVIDIA RTX A6000, pci bus id: 0000:d9:00.0, compute capability: 8.6
2023-02-15 16:04:49.002120: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:357] MLIR V1 optimization pass is not e

EqualizedOdds 0.6900074019245004
EqualizedOdds 0.6947446336047373
EqualizedOdds 0.6945965951147298
0.6934122871946706
0.6888971132494448
data/datasets/publiccov_ny/synthetic/gender_flip_labels0.1_version1/


2023-02-15 16:11:16.003455: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46704 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:b1:00.0, compute capability: 8.6
2023-02-15 16:11:16.003759: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 46704 MB memory:  -> device: 1, name: NVIDIA RTX A6000, pci bus id: 0000:d9:00.0, compute capability: 8.6


EqualizedOdds 0.6924500370096225
EqualizedOdds 0.6910436713545521
0.691339748334567
0.6901554404145078
data/datasets/publiccov_ny/synthetic/gender_flip_labels0.1_version2/


...

2023-02-16 00:09:32.793044: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46704 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:b1:00.0, compute capability: 8.6
2023-02-16 00:09:32.793673: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 46704 MB memory:  -> device: 1, name: NVIDIA RTX A6000, pci bus id: 0000:d9:00.0, compute capability: 8.6


EqualizedOdds 0.8439805923380168
EqualizedOdds 0.8388759729101385
0.8409986859395532
0.8305872839381381
data/datasets/publiccov_tx/synthetic/gender_flip_labels0.4_version3/


2023-02-16 00:13:45.215650: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46704 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:b1:00.0, compute capability: 8.6
2023-02-16 00:13:45.215991: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 46704 MB memory:  -> device: 1, name: NVIDIA RTX A6000, pci bus id: 0000:d9:00.0, compute capability: 8.6


EqualizedOdds 0.8425149095319924
EqualizedOdds 0.8379156979682604
0.841453553017285
0.8195188517133326
data/datasets/publiccov_tx/synthetic/gender_flip_labels0.4_version4/


2023-02-16 00:18:04.264073: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46704 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:b1:00.0, compute capability: 8.6
2023-02-16 00:18:04.268453: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 46704 MB memory:  -> device: 1, name: NVIDIA RTX A6000, pci bus id: 0000:d9:00.0, compute capability: 8.6


In [ ]:
# from multiprocessing.pool import ThreadPool as Pool

# pool = Pool(10)
# for result in pool.imap_unordered(task, paths):
#     print(result)